In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pyabf
import glob
import seaborn as sns
import pandas as pd
import os
import scipy.stats as stats

In [17]:
def baseline_sweep(abf):
    #baseline based on the slope
    baseline = np.mean(abf.sweepY)
    data_out = abf.sweepY - baseline
    return data_out


In [18]:
MAIN_FOLDER = "Z:\\Molsrv\\Julia\\Data\\Opto\\Opto Perifornical for Grant_2025\\ALL FILES"
xlsx_files = glob.glob(MAIN_FOLDER + "\\*.xlsx")
all_results = {}
PRE_TIME = (1.027*1000, 1.047*1000)
POST_TIME = (1.047*1000, 1.062*1000)
col_map = {0: "sweep", 4: "event_time", 7:"peak_amp"}
col_map_inv = {v: k for k, v in col_map.items()}
RAW_DATA = {}
for fe in xlsx_files:
    if "~$" in fe:
        continue
    df = pd.read_excel(fe, header=None)
    
    df.rename(columns=col_map, inplace=True)
    print(f"Processing {fe} with {len(df)} rows")
    #get the filename without extension
    filename = os.path.splitext(os.path.basename(fe))[0]
    



    if os.path.exists(os.path.join(MAIN_FOLDER, filename + ".abf")):
        abf = pyabf.ABF(os.path.join(MAIN_FOLDER, filename + ".abf"), loadData=False)
        print(f"Loaded {abf.abfID} with {abf.sweepCount} sweeps")
        #actually load it
        abf = pyabf.ABF(os.path.join(MAIN_FOLDER, filename + ".abf"))
        



    else: 
        print(f"!!!!!       ABF file for {filename} not found, skipping !!!!!!!!!!!!!!!!!!!!!!!")
        break
    if df.empty:
        #fill everything with zeros
        unested_results = {
            "filename": filename,
            "abfID": None,
            "sweepCount": 0,
            "pre_count_mean": 0,
            "post_count_mean": 0,
            "pre_count_std": 0,
            "post_count_std": 0,
        }
        unested_results.update({'p': 1.0, 'stat': 0})
        pre_columns = {f"PRE_{i}": 0 for i in range(abf.sweepCount)}
        post_columns = {f"POST_{i}": 0 for i in range(abf.sweepCount)}


        all_results[fe] = unested_results
        RAW_DATA[fe] = np.array([])
        continue
    results = {"Pre": [], "Post": [], 'max': []}
    for sweep in np.arange(abf.sweepCount):
        #get that sweep from the df
        sweep_df = df[df["sweep"] == sweep]
        #filter events smaller than -5 pA
        sweep_df = sweep_df[sweep_df["peak_amp"] < -5]
        abf.setSweep(sweep)
        idx_start = np.argmin(np.abs(abf.sweepX - POST_TIME[0]/1000))
        idx_end = np.argmin(np.abs(abf.sweepX - POST_TIME[1]/1000))
        sweep_baselined = baseline_sweep(abf)
        if not sweep_df.empty:
            event_time = sweep_df["event_time"].values
            #get the count of events pre and post
            pre_count = np.sum(np.logical_and(event_time >= PRE_TIME[0], event_time < PRE_TIME[1]))
            post_count = np.sum(np.logical_and(event_time >= POST_TIME[0], event_time < POST_TIME[1]))
            results["Pre"].append(pre_count)
            results["Post"].append(post_count)
            #get the peak (negative amp)
            max_peak = np.min(sweep_baselined[idx_start:idx_end])
            
            results["max"].append(max_peak)


        else:
            results["Pre"].append(0)
            results["Post"].append(0)
            max_peak = np.min(sweep_baselined[idx_start:idx_end])
            results["max"].append(max_peak)

    #run wilcoxon test
    #special case if all zero
    if all(x == 0 for x in results["Pre"]) and all(x == 0 for x in results["Post"]):
        print(f"All zero counts for {fe}, skipping Wilcoxon test")
        stat, p = 0, 1.0
    else:
        stat, p = stats.wilcoxon(results["Pre"], results["Post"], zero_method="pratt", alternative="less")
    print(f"Wilcoxon test results for {fe}: stat={stat}, p={p}")
    results["stat"] = stat
    results["p"] = p

    unested_results = {k: v for k, v in results.items() if k != "Pre" and k != "Post"}
    unested_results.update({
        "filename": filename,
        "abfID": abf.abfID,
        "sweepCount": abf.sweepCount,
        "pr_count_mean": np.mean(results["Pre"]),
        "pos_count_mean": np.mean(results["Post"]),
        "pr_count_std": np.std(results["Pre"]),
        "pos_count_std": np.std(results["Post"]),
        "pr_count_tot": np.sum(results["Pre"]),
        "pos_count_tot": np.sum(results["Post"]),
        "mean_peak": np.mean(results["max"]),


    })
    pre_columns = {f"PRE_{i}": v for i, v in enumerate(results["Pre"])}
    post_columns = {f"POST_{i}": v for i, v in enumerate(results["Post"])}
    all_results[fe] = {**unested_results, **pre_columns, **post_columns}

    RAW_DATA[fe] = df[["sweep", "event_time"]].to_numpy()



Processing Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0003.xlsx with 20 rows
Loaded 2025_08_06_0003 with 18 sweeps


c:\Users\SMest\.conda\envs\pyAPisolation\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Wilcoxon test results for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0003.xlsx: stat=62.0, p=0.7928919108787373
Processing Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0007.xlsx with 80 rows
Loaded 2025_08_06_0007 with 19 sweeps


c:\Users\SMest\.conda\envs\pyAPisolation\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Wilcoxon test results for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0007.xlsx: stat=104.5, p=0.9166174601411015
Processing Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0018.xlsx with 1 rows
Loaded 2025_08_06_0018 with 5 sweeps
All zero counts for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0018.xlsx, skipping Wilcoxon test
Wilcoxon test results for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0018.xlsx: stat=0, p=1.0
Processing Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\2025_08_06_0054.xlsx with 40 rows
Loaded 2025_08_06_0054 with 18 sweeps


...

c:\Users\SMest\.conda\envs\pyAPisolation\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Wilcoxon test results for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\25813066.xlsx: stat=0.0, p=0.07864960352514258
Processing Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\25813073.xlsx with 3 rows
Loaded 25813073 with 25 sweeps


c:\Users\SMest\.conda\envs\pyAPisolation\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Wilcoxon test results for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\25813073.xlsx: stat=0.0, p=0.15865525393145707
Processing Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\25813077.xlsx with 14 rows
Loaded 25813077 with 25 sweeps
Wilcoxon test results for Z:\Molsrv\Julia\Data\Opto\Opto Perifornical for Grant_2025\ALL FILES\25813077.xlsx: stat=69.0, p=0.6726395769907115


c:\Users\SMest\.conda\envs\pyAPisolation\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [19]:
# Combine all results into a single DataFrame
combined_results = pd.DataFrame.from_dict(all_results, orient="index")

#make sure the pre_columns come before the post_columns
pre_columns = [col for col in combined_results.columns if col.startswith("PRE_")]
post_columns = [col for col in combined_results.columns if col.startswith("POST_")]
#non_pre_columns
non_pre_columns = [col for col in combined_results.columns if col not in pre_columns and col not in post_columns]
# reorder the columns
combined_results = combined_results[non_pre_columns + pre_columns + post_columns]

combined_results.to_csv(os.path.join(MAIN_FOLDER, "combined_results.csv"))

In [20]:
prev_event_count = 0
plt.close()
plt.figure(figsize=(6, 6))
for i, (key, event_time) in enumerate(RAW_DATA.items()):
    p_val = combined_results.loc[key, "p"]
    c = 'r' if p_val < 0.05 else 'k'
    if len(event_time) > 0:
        plt.scatter(event_time[:, 1], event_time[:, 0] + (i*25), alpha=0.5, marker='|', label='Event Times', c=c)
    #check if the date changes
    
plt.xlim([950, 1100])
plt.axvline(x=POST_TIME[0], color='g', linestyle='--', label='stim')
plt.show()

In [21]:


#load in metadata
metadata = pd.read_excel("Z:\\Molsrv\\Julia\\Data\\Opto\\Opto Perifornical for Grant_2025\\FILENAMES_Opto Perifornical_FOR_YEHOR.xlsx")



FileNotFoundError: [Errno 2] No such file or directory: 'Z:\\Molsrv\\Julia\\Data\\Opto\\Opto Perifornical for Grant_2025\\FILENAMES_Opto Perifornical_FOR_YEHOR.xlsx'

In [ ]:
print(metadata.tail())

     Cell ID      Date Mouse ID Sex Condition Slice ID Well#     Filled?  \
101  5_SM_14       NaN    860-9   F       NaN      3-R   NaN  Not filled   
102  5_SM_15       NaN    860-9   F       NaN      2-R   NaN  Not filled   
103  5_SM_16       NaN    860-9   F       NaN      2-R   NaN  Not filled   
104  5_SM_17       NaN    860-9   F       NaN      2-R   NaN  Not filled   
105  2_SM_18  250807.0    863-9   M        KD      2-R   NaN  Not filled   

                Area Exerimenter  ...  PRE_21 PRE_22 PRE_23  PRE_24  POST_19  \
101       PF-lateral          SM  ...     0.0    0.0    0.0     0.0      0.0   
102   PF-dorsomedial          SM  ...     0.0    0.0    0.0     0.0      0.0   
103        PF-medial          SM  ...     0.0    0.0    0.0     0.0      1.0   
104  PF-dorsolateral          SM  ...     0.0    0.0    0.0     0.0      0.0   
105  PF-ventromedial          SM  ...     0.0    0.0    0.0     0.0      0.0   

     POST_20 POST_21 POST_22 POST_23 POST_24  
101      0.0   

In [ ]:
#join based on the "PP (20 ms)" column in metadata, and index in combined_results
combined_results_meta = combined_results.set_index("filename").join(metadata.set_index("PP (20 ms)"))

ValueError: columns overlap but no suffix specified: Index(['stat', 'p', 'abfID', 'sweepCount', 'pre_count_mean', 'post_count_mean',
       'pre_count_std', 'post_count_std', 'PRE_0', 'PRE_1', 'PRE_2', 'PRE_3',
       'PRE_4', 'PRE_5', 'PRE_6', 'PRE_7', 'PRE_8', 'PRE_9', 'PRE_10',
       'PRE_11', 'PRE_12', 'PRE_13', 'PRE_14', 'PRE_15', 'PRE_16', 'PRE_17',
       'PRE_18', 'PRE_19', 'PRE_20', 'PRE_21', 'PRE_22', 'PRE_23', 'PRE_24',
       'POST_0', 'POST_1', 'POST_2', 'POST_3', 'POST_4', 'POST_5', 'POST_6',
       'POST_7', 'POST_8', 'POST_9', 'POST_10', 'POST_11', 'POST_12',
       'POST_13', 'POST_14', 'POST_15', 'POST_16', 'POST_17', 'POST_18',
       'POST_19', 'POST_20', 'POST_21', 'POST_22', 'POST_23', 'POST_24'],
      dtype='object')

In [ ]:
combined_results_meta.to_csv(os.path.join(MAIN_FOLDER, "combined_results_with_metadata.csv"))